# load modules

In [1]:
%%capture
%load_ext autoreload
%autoreload 2
import sys
import os
import gc
import psutil
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
import bokeh.plotting as bpl
import dask.array as da
import pandas as pd
import dask
import datashader as ds
import itertools as itt
import papermill as pm
import ast
import functools as fct
from holoviews.operation.datashader import datashade, regrid, dynspread
from datashader.colors import Sets1to3
from dask.diagnostics import ProgressBar
from IPython.core.display import display, HTML
from dask.distributed import Client, progress, LocalCluster, fire_and_forget
import numpy as np

# define directories in list

In [6]:
#folders = [
#    ""
#]

data_root = r'/media/share/csstorage/Denise/backup_8_1_18/G-Time 1 Data/G123/'
write_root = r'/home/wmau/GTime/GTime1/G123/'

# 8 sessions
read_paths = []
folders = []
for n in np.arange(1,9):
    intermediate_path = os.path.join(data_root, str(n))     # e.g., .../G123/1/
    session_folder = os.listdir(intermediate_path)[0]       # e.g., H9_M57_S37
    
    read_paths.append(os.path.join(intermediate_path, session_folder))  # Folder containing videos
    
    # Folder to write to
    write_folder = os.path.join(write_root, str(n))
    os.mkdir(write_folder)
    folders.append(write_folder)                     

# specify paramaters

In [ ]:
#Set up Initial Basic Parameters#
minian_path = "."
subset = dict(frame=slice(0,30*60*8))
subset_mc = None
interactive = False
output_size = 100
param_save_minian = {
    'dpath': "",
    'fname': 'minian',
    'backend': 'zarr',
    'meta_dict': dict(session_id=-1, session=-3, animal=-4),
    'overwrite': False}

#Pre-processing Parameters#
param_load_videos = {
    'pattern': 'msCam[0-9]+\.avi$',
    'dtype': np.float32,
    'downsample': dict(frame=3,height=1,width=1),
    'downsample_strategy': 'subset'}
param_glow_removal = {
    'method': 'uniform',
    'wnd': 51}
param_first_denoise = {
    'method': 'median',
    'ksize': 5}
param_second_denoise = {
    'method': 'gaussian',
    'sigmaX': 0,
    'ksize': (5, 5)}

#Motion Correction Parameters#
subset_mc = None
param_estimate_shift = {
    'dim': 'frame',
    'on': 'first',
    'pad_f': 1,
    'pct_thres': 99.99}

#Background Removal Parameters#
param_background_removal = {
    'method': 'tophat',
    'wnd': 10}

#Initialization Parameters#
param_seeds_init = {
    'wnd_size': 600,
    'method': 'rolling',
    'stp_size': 600,
    'nchunk': 100,
    'max_wnd': 5}
param_gmm_refine = {
    'q': (0.1, 99.9),
    'n_components': 2,
    'valid_components': 1,
    'mean_mask': True}
param_pnr_refine = {
    'noise_freq': 0.1,
    'thres': 1,
    'med_wnd': None}
param_ks_refine = {
    'sig': 0.05}
param_seeds_merge = {
    'thres_dist': 3,
    'thres_corr': 0.7,
    'noise_freq': .06}
param_initialize = {
    'thres_corr': 0.5,
    'wnd': 5}

#CNMF Parameters#
param_get_noise = {
    'noise_range': (0.1, 0.5),
    'noise_method': 'logmexp'}
param_first_spatial = {
    'dl_wnd': 3,
    'sparse_penal': 0.5,
    'update_background': False,
    'post_scal': True,
    'zero_thres': 'eps'}
param_first_temporal = {
    'noise_freq': 0.1,
    'sparse_penal': 5,
    'p': 2,
    'add_lag': 10,
    'use_spatial': False,
    'jac_thres': 0.2,
    'zero_thres': 1e-8,
    'max_iters': 500,
    'use_smooth': True,
    'scs_fallback': False,
    'post_scal': True}
param_first_merge = {
    'thres_corr': 0.9}
param_second_spatial = {
    'dl_wnd': 3,
    'sparse_penal': 0.1,
    'update_background': False,
    'post_scal': True,
    'zero_thres': 'eps'}
param_second_temporal = {
    'noise_freq': 0.1,
    'sparse_penal': 5,
    'p': 2,
    'add_lag': 10,
    'use_spatial': False,
    'jac_thres': 0.2,
    'zero_thres': 1e-8,
    'max_iters': 500,
    'use_smooth': True,
    'scs_fallback': False,
    'post_scal': True}
param_second_merge = {
    'thres_corr': 0.9}

# load minian

In [ ]:
%%capture
sys.path.append(minian_path)
from minian.utilities import load_params, load_videos, scale_varr, scale_varr_da, save_variable, open_minian, save_minian, handle_crash, get_optimal_chk, downsamplex
from minian.preprocessing import remove_brightspot, gradient_norm, denoise, remove_background, stripe_correction
from minian.motion_correction import estimate_shift_fft, apply_shifts, interpolate_frame, mask_shifts
from minian.initialization import seeds_init, gmm_refine, pnr_refine, intensity_refine, ks_refine, seeds_merge, initialize
from minian.cnmf import psd_welch, psd_fft, get_noise, update_spatial, compute_trace, update_temporal, unit_merge, smooth_sig
from minian.visualization import VArrayViewer, CNMFViewer, generate_videos, visualize_preprocess, visualize_seeds, visualize_gmm_fit, visualize_spatial_update, visualize_temporal_update, roi_draw


# module initialization

In [ ]:
para_norm_list = ['meta_dict', 'chunks', 'subset', 'subset_mc']
for par_key in list(globals().keys()):
    if par_key in para_norm_list or par_key.startswith('param_'):
        globals()[par_key] = load_params(globals()[par_key])

hv.notebook_extension('bokeh', width=100)
pbar = ProgressBar(minimum=2)
pbar.register()

# loop through folders

In [ ]:
for read_folder, write_folder in zip(read_paths, folders):

    #reset path
    dpath = os.path.abspath(read_folder)
    param_save_minian['dpath'] = write_folder


    #load videos
    varr = load_videos(dpath, **param_load_videos)
    varr_ref = varr.sel(subset)
    if subset:
        for sub_key in subset.keys():
            varr_reg = varr_ref.dropna(sub_key)
    chk = get_optimal_chk(varr_ref, [('frame',), ('height', 'width')])


    #preprocessing
    varr_ref = remove_background(varr_ref, **param_glow_removal)
    varr_ref = denoise(varr_ref, **param_first_denoise)
    varr_ref = denoise(varr_ref, **param_second_denoise)


    #motion correction
    res = estimate_shift_fft(varr_ref.sel(subset_mc), **param_estimate_shift)
    shifts = res.sel(variable = ['height', 'width'])
    corr = res.sel(variable='corr')
    shifts = shifts.compute()
    save_minian(shifts.rename('shifts'), **param_save_minian)
    varr_mc = apply_shifts(varr_ref, shifts)
    varr_mc = varr_mc.ffill('height').bfill('height').ffill('width').bfill('width')


    #background subtraction
    Y = remove_background(varr_mc, **param_background_removal)
    Y = scale_varr(Y, pre_compute=False)


    #spatially downsample prior to cnmf?
    #varr_mc = downsamplex(varr_mc, downsample=dict(width=2,height=2))
    varr_mc = varr_mc.chunk(chk)
    #Y = downsamplex(Y, downsample=dict(width=2,height=2))
    Y = Y.chunk(chk)
    save_minian(varr_mc.rename('org'), **param_save_minian)
    save_minian(Y.rename('Y'), **param_save_minian)


    #initialization
    minian = open_minian(param_save_minian['dpath'],
                         fname=param_save_minian['fname'],
                         backend=param_save_minian['backend'])
    Y = minian['Y']
    max_proj = Y.max('frame').compute()
    Y_flt = Y.stack(spatial=['height', 'width'])
    seeds = seeds_init(Y, **param_seeds_init)
    seeds, pv, gmm= gmm_refine(Y_flt, seeds, **param_gmm_refine)
    seeds, pnr, gmm = pnr_refine(Y_flt, seeds[seeds['mask_gmm']].copy(), **param_pnr_refine)
    seeds = ks_refine(Y_flt, seeds[seeds['mask_pnr']], **param_ks_refine)
    seeds_final = seeds[seeds['mask_ks']].reset_index(drop=True)
    seeds_mrg = seeds_merge(Y_flt, seeds_final, **param_seeds_merge)
    A, C, b, f = initialize(Y, seeds_mrg[seeds_mrg['mask_mrg']], **param_initialize)
    save_minian(A.rename('A_init').rename(unit_id='unit_id_init'), **param_save_minian)
    save_minian(C.rename('C_init').rename(unit_id='unit_id_init'), **param_save_minian)
    save_minian(b.rename('b_init'), **param_save_minian)
    save_minian(f.rename('f_init'), **param_save_minian)


    #CNMF
    minian = open_minian(param_save_minian['dpath'],
                         fname=param_save_minian['fname'],
                         backend=param_save_minian['backend'])
    Y = minian['Y']
    A_init = minian['A_init'].rename(unit_id_init='unit_id')
    C_init = minian['C_init'].rename(unit_id_init='unit_id')
    b_init = minian['b_init']
    f_init = minian['f_init']
    psd = psd_fft(Y)
    sn_spatial = get_noise(psd, **param_get_noise).persist()
    #first spatial
    A_spatial, b_spatial, C_spatial, f_spatial = update_spatial(
        Y, A_init, b_init, C_init, f_init, sn_spatial, **param_first_spatial)
    #first tempopral
    YrA, C_temporal, S_temporal, B_temporal, C0_temporal, sig_temporal, g_temporal, scale = update_temporal(
        Y, A_spatial, b_spatial, C_spatial, f_spatial, sn_spatial, **param_first_temporal)
    A_temporal = A_spatial.sel(unit_id = C_temporal.coords['unit_id'])
    #first merge
    A_mrg, sig_mrg, add_list = unit_merge(A_temporal, sig_temporal, [S_temporal, C_temporal], **param_first_merge)
    S_mrg, C_mrg = add_list[:]
    #second spatial
    A_spatial_it2, b_spatial_it2, C_spatial_it2, f_spatial_it2 = update_spatial(
        Y, A_mrg, b_spatial, sig_mrg, f_spatial, sn_spatial, **param_second_spatial)
    #second temporal
    YrA, C_temporal_it2, S_temporal_it2, B_temporal_it2, C0_temporal_it2, sig_temporal_it2, g_temporal_it2, scale_temporal_it2 = update_temporal(
        Y, A_spatial_it2, b_spatial_it2, C_spatial_it2, f_spatial_it2, sn_spatial, **param_second_temporal)
    A_temporal_it2 = A_spatial_it2.sel(unit_id=C_temporal_it2.coords['unit_id'])
    #second merge
    A_mrg_it2, C_mrg_it2, add_list = unit_merge(A_temporal_it2, C_temporal_it2, [S_temporal_it2, C0_temporal_it2, g_temporal_it2, B_temporal_it2], **param_second_merge)
    S_mrg_it2, C0_mrg_it2, g_mrg_it2, B_mrg_it2 = add_list[:]
    #save
    save_minian(A_mrg_it2.rename('A'), **param_save_minian)
    save_minian(C_mrg_it2.rename('C'), **param_save_minian)
    save_minian(S_mrg_it2.rename('S'), **param_save_minian)
    save_minian(g_mrg_it2.rename('g'), **param_save_minian)
    save_minian(C0_mrg_it2.rename('C0'), **param_save_minian)
    save_minian(B_mrg_it2.rename('BL'), **param_save_minian)
    save_minian(b_spatial_it2.rename('b'), **param_save_minian)
    save_minian(f_spatial_it2.rename('f'), **param_save_minian)


    #generate video
    minian = open_minian(param_save_minian['dpath'],
                         fname=param_save_minian['fname'],
                         backend=param_save_minian['backend'])
    generate_videos(
        minian, os.path.join(dpath, param_save_minian['fname'] + ".mp4"))